In [26]:
%reset -f

import pandas as pd
import numpy as np
import random
import scipy as sci
import sympy as sym
import math
import matplotlib
import matplotlib.pyplot as plt
import itertools
import gurobipy as gp
import pickle

from numpy import sqrt, exp, log, sin, cos, pi, polyfit, polyval, vander
from numpy.linalg import inv, pinv, det, matrix_rank, cond, lstsq, solve
from scipy.integrate import solve_ivp
from gurobipy import GRB
from pyinstrument import Profiler
from numba import jit
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

plt.style.use('default')                                    # 使用默认风格
plt.rcParams['figure.facecolor'] = 'white'                  # 将图形的背景颜色设置为白色
plt.rcParams['font.family'] = 'YaHei Consolas Hybrid'       # 字体确认
plt.rcParams['font.size'] = 12                              # 字体大小
plt.rcParams['axes.unicode_minus'] = False                  # 正常显示负号
%matplotlib inline
# matplotlib.use('TkAgg')

from Algorithms.Algorithm import *
from Algorithms.Params import *


In [27]:
## 仿真测试一

class Args:
    def __init__(self, n,m,PT,MT,ni,parallel_dict,need2quick,means_m=1,
                 pop_size=100,gene_size=500,
                 pc_max=0.8,pm_max=0.75,
                 pc_min=0.3,pm_min=0.3,
                 p_GS=0.5,p_LS=0.3,p_RS=0.2,
                 T=10,H=16,N_elite=4):
        self.n=n
        self.m=m
        self.Processing_Time=PT
        self.Processing_Machine=MT
        self.O_num=ni
        self.parallel_dict = parallel_dict
        self.need2quick = need2quick
        self.means_m=means_m
        self.pop_size=pop_size
        self.gene_size=gene_size
        self.pc_max=pc_max
        self.pm_max=pm_max
        self.pc_min=pc_min
        self.pm_min=pm_min
        self.p_GS=p_GS
        self.p_LS=p_LS
        self.p_RS=p_RS
        self.T=T
        self.H=H
        self.N_elite=N_elite
        
        
## 读取原始内容

D1 = pd.read_excel('工序机床表.xlsx')
D2 = pd.read_excel('工序耗时表.xlsx')


# 工件种类
n = D1.shape[0]
# 数控机床数目
m = 8
# 每种工件的工序数目
ni = [0 for i in range(n)]
# 每一个工序对应机床的集合
precess_Machine = []
# 每一个工序在precess_Machine对应的时间
precess_time = []
# means_m大于1表示用于解决FJSP问题
means_m = 2

# 并行关系字典：工件编号从0开始，工序数从0开始，机器编号从1开始
parallel_dict = {(0,1):(0,2),(0,2):(0,1),
                 (3,1):(3,2),(3,2):(3,1),
                 (4,1):(4,2),(4,2):(4,1),
                 (5,2):(5,3),(5,3):(5,2),
                 (6,2):(6,3),(6,3):(6,2),
                 (7,2):(7,3),(7,3):(7,2),
                 }

# 加急列表(0号开始)
need2quick = [0,2,4,6,8]

# 数据处理

for i in range(n):
    precess_Machine.append([])
    precess_time.append([])
for ind, row in D1.iterrows():
    for col in range(1, len(row)):
        curtime = D2.iloc[ind, col]
        pm = []
        pt = []
        if pd.isna(row[col]):
            break
        ni[row[0]] += 1
        ele = row[col].split(',')
        for e in ele:
            pm.append(int(e))
            pt.append(float(curtime))
        
        precess_Machine[ind].append(pm)
        precess_time[ind].append(pt)


## 增加新的工件和机器

# 使用固定种子生成同一批订单
random.seed(1145141919810)

add_workpieces = 11
add_machine = 4

# 新增工件的工序数的上下界
operation_lb = 2
operation_ub = 6

# 每一个工序可选机器数目的上下界
chooseMac_lb = 1
chooseMac_ub = 5

# 每一个工序的加工时间的上下界
precessTime_lb = 3000
precessTime_ub = 25000

# -----------------------------------

# 构建可选机器列表
MachineLst = list(range(1,m + 1))
newMachineLst = list(range(m + 1, m + add_machine + 1))
add_ni = [0 for i in range(add_workpieces)]

for i in range(add_workpieces):
    # 对于每一个新的工件，随机一个工序数目workoperations
    this_workpiece = []
    this_worktime = []
    operations = random.randint(operation_lb, operation_ub)
    add_ni[i] = operations
    
    for j in range(operations):

        chooseMacs = random.randint(chooseMac_lb, chooseMac_ub)
        # 新机器多选一点
        choose_newMacs = int(np.floor(chooseMacs * 0.6))
        chooseMacs_oriMacs = chooseMacs - choose_newMacs
        # 从机器列表中随机抽取随机个数的样本
        curMachineSet = random.sample(MachineLst, chooseMacs_oriMacs)
        curMachineSet.extend(random.sample(newMachineLst, choose_newMacs))
        curMachineSet = sorted(curMachineSet)
        
        # 生成工序在每一台机器上的耗时
        cost = random.randint(precessTime_lb, precessTime_ub)
        curTimeSet = [cost for _ in range(len(curMachineSet))]
        this_workpiece.append(curMachineSet)
        this_worktime.append(curTimeSet)
    
    # 生成并行(有一半的概率存在并行)
    if operations >= 3 and random.random() > 0.5:
        # 如果工序数大于等于3，就添加一处并行
        parallel_idx = random.randint(1, operations - 2) # 0 [ 1  2  3  4  5 ] 6  
        # 将并行信息添加到字典
        # 第n + i + 1号工件的第parallel_idx工序和parallel_idx + 1工序存在并行关系
        parallel_dict[(n + i, parallel_idx)] = (n + i, parallel_idx + 1)
        parallel_dict[(n + i, parallel_idx + 1)] = (n + i, parallel_idx)
    
    precess_Machine.append(this_workpiece)
    precess_time.append(this_worktime)


n += add_workpieces
m += add_machine
ni.extend(add_ni)       

args = Args(n,m,precess_time,precess_Machine,ni,parallel_dict,need2quick,means_m)

Algo = Algorithms(args)

# 输出信息
print('工件种类数目：',n)
print('机器数目：',m)
print('每个工件的工序数：',ni)
print('\n')
for i in range(len(precess_Machine)):
    print(f"第{i + 1}号工件的所有工序对应机器集合",precess_Machine[i], "\n对应的加工时间",precess_time[i])

print("\n并列关系：\n\n")
for key,val in parallel_dict.items():
    if key[1] < val[1]:
        print(f"{key[0] + 1}号工件的{key[1] + 1}和{val[1] + 1}号工序存在并行关系")
        
# 输出excel



工件种类数目： 20
机器数目： 12
每个工件的工序数： [4, 3, 3, 3, 4, 4, 4, 4, 2, 6, 4, 3, 4, 4, 3, 2, 2, 2, 4, 6]


第1号工件的所有工序对应机器集合 [[2, 3, 4], [2, 3, 4], [6, 7], [2, 3, 4]] 
对应的加工时间 [[11553.0, 11553.0, 11553.0], [5769.0, 5769.0, 5769.0], [4657.0, 4657.0], [2616.0, 2616.0, 2616.0]]
第2号工件的所有工序对应机器集合 [[6, 7], [6, 7], [2, 3, 4]] 
对应的加工时间 [[17380.0, 17380.0], [19770.0, 19770.0], [4905.0, 4905.0, 4905.0]]
第3号工件的所有工序对应机器集合 [[5, 6, 7, 8], [2, 3, 4], [5, 6, 7, 8]] 
对应的加工时间 [[8410.0, 8410.0, 8410.0, 8410.0], [15885.0, 15885.0, 15885.0], [3902.0, 3902.0, 3902.0, 3902.0]]
第4号工件的所有工序对应机器集合 [[5, 6, 7], [2, 3, 4], [2, 3, 4]] 
对应的加工时间 [[10370.0, 10370.0, 10370.0], [10232.0, 10232.0, 10232.0], [4081.0, 4081.0, 4081.0]]
第5号工件的所有工序对应机器集合 [[1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4], [2, 3, 4]] 
对应的加工时间 [[11655.0, 11655.0, 11655.0, 11655.0], [7043.0, 7043.0, 7043.0, 7043.0], [14355.0, 14355.0, 14355.0, 14355.0], [23410.0, 23410.0, 23410.0]]
第6号工件的所有工序对应机器集合 [[5, 6, 7, 8], [5, 6, 7, 8], [2, 3, 4], [1, 2, 3, 4]] 
对应的加工时间 [[6480.0,

In [28]:
## 第二次仿真测试

class Args:
    def __init__(self, n,m,PT,MT,ni,parallel_dict,need2quick,means_m=1,
                 pop_size=100,gene_size=500,
                 pc_max=0.8,pm_max=0.75,
                 pc_min=0.3,pm_min=0.3,
                 p_GS=0.5,p_LS=0.3,p_RS=0.2,
                 T=10,H=16,N_elite=4):
        self.n=n
        self.m=m
        self.Processing_Time=PT
        self.Processing_Machine=MT
        self.O_num=ni
        self.parallel_dict = parallel_dict
        self.need2quick = need2quick
        self.means_m=means_m
        self.pop_size=pop_size
        self.gene_size=gene_size
        self.pc_max=pc_max
        self.pm_max=pm_max
        self.pc_min=pc_min
        self.pm_min=pm_min
        self.p_GS=p_GS
        self.p_LS=p_LS
        self.p_RS=p_RS
        self.T=T
        self.H=H
        self.N_elite=N_elite


## 增加新的工件和机器
n = 0
m = 0
means_m = 2
ni = []
parallel_dict = {}
precess_Machine = []
precess_time = []
need2quick = []

# 使用固定种子生成同一批订单
random.seed(1145141919810)

add_workpieces = 20
add_machine = 10

# 新增工件的工序数的上下界
operation_lb = 2
operation_ub = 6

# 每一个工序可选机器数目的上下界
chooseMac_lb = 1
chooseMac_ub = 5

# 每一个工序的加工时间的上下界
precessTime_lb = 3000
precessTime_ub = 25000

# -----------------------------------

# 构建可选机器列表
MachineLst = list(range(1,m + 1))
newMachineLst = list(range(m + 1, m + add_machine + 1))
add_ni = [0 for i in range(add_workpieces)]

for i in range(add_workpieces):
    # 对于每一个新的工件，随机一个工序数目workoperations
    this_workpiece = []
    this_worktime = []
    operations = random.randint(operation_lb, operation_ub)
    add_ni[i] = operations
    
    for j in range(operations):

        chooseMacs = random.randint(chooseMac_lb, chooseMac_ub)
        # 新机器多选一点
        curMachineSet = random.sample(newMachineLst, chooseMacs)
        curMachineSet = sorted(curMachineSet)
        
        # 生成工序在每一台机器上的耗时
        cost = random.randint(precessTime_lb, precessTime_ub)
        curTimeSet = [cost for _ in range(len(curMachineSet))]
        this_workpiece.append(curMachineSet)
        this_worktime.append(curTimeSet)
    
    # 生成并行(有一半的概率存在并行)
    if operations >= 3 and random.random() > 0.5:
        # 如果工序数大于等于3，就添加一处并行
        parallel_idx = random.randint(1, operations - 2) # 0 [ 1  2  3  4  5 ] 6  
        # 将并行信息添加到字典
        # 第n + i + 1号工件的第parallel_idx工序和parallel_idx + 1工序存在并行关系
        parallel_dict[(n + i, parallel_idx)] = (n + i, parallel_idx + 1)
        parallel_dict[(n + i, parallel_idx + 1)] = (n + i, parallel_idx)
    
    precess_Machine.append(this_workpiece)
    precess_time.append(this_worktime)


n += add_workpieces
m += add_machine
ni.extend(add_ni)       

args = Args(n,m,precess_time,precess_Machine,ni,parallel_dict,need2quick,means_m)

Algo = Algorithms(args)

# 输出信息
print('工件种类数目：',n)
print('机器数目：',m)
print('每个工件的工序数：',ni)
print('\n')
for i in range(len(precess_Machine)):
    print(f"第{i + 1}号工件的所有工序对应机器集合",precess_Machine[i], "\n对应的加工时间",precess_time[i])

print("\n并列关系：\n\n")
for key,val in parallel_dict.items():
    if key[1] < val[1]:
        print(f"{key[0] + 1}号工件的{key[1] + 1}和{val[1] + 1}号工序存在并行关系")

工件种类数目： 20
机器数目： 10
每个工件的工序数： [6, 5, 5, 3, 4, 3, 2, 5, 6, 5, 5, 2, 3, 2, 2, 2, 3, 5, 2, 4]


第1号工件的所有工序对应机器集合 [[4], [4, 9], [5, 9], [5, 10], [2, 5, 7, 8], [3, 5, 7]] 
对应的加工时间 [[11724], [14135, 14135], [18773, 18773], [13483, 13483], [18605, 18605, 18605, 18605], [4168, 4168, 4168]]
第2号工件的所有工序对应机器集合 [[2, 6, 8], [2, 3, 9], [7, 8], [1, 2, 5], [3]] 
对应的加工时间 [[17139, 17139, 17139], [24073, 24073, 24073], [5635, 5635], [12884, 12884, 12884], [18501]]
第3号工件的所有工序对应机器集合 [[2, 4, 5, 7, 8], [2, 3, 4, 7, 9], [1, 5, 10], [9], [9]] 
对应的加工时间 [[15987, 15987, 15987, 15987, 15987], [15464, 15464, 15464, 15464, 15464], [16835, 16835, 16835], [12078], [18893]]
第4号工件的所有工序对应机器集合 [[3, 4, 6, 8, 10], [1, 3, 6], [2]] 
对应的加工时间 [[12555, 12555, 12555, 12555, 12555], [13896, 13896, 13896], [24577]]
第5号工件的所有工序对应机器集合 [[1, 4, 6, 7, 9], [1, 2, 3, 8], [1, 3, 8], [3, 5, 6, 10]] 
对应的加工时间 [[12825, 12825, 12825, 12825, 12825], [8683, 8683, 8683, 8683], [19266, 19266, 19266], [20716, 20716, 20716, 20716]]
第6号工件的所有工序对应机器集合 [[4

In [ ]:
EP,ObjInfo,pc,pm=Algo.NSGA_main()

In [ ]:
with open('data.pickle', 'rb') as f:
    data = pickle.load(f)

# 从data字典中获取EP、ObjInfo、pc和pm
EP = data['EP']
ObjInfo = data['ObjInfo']
pc = data['pc']
pm = data['pm']

In [ ]:
## 按照优先级排序
fitness = []
for i in range(len(EP)):
    fitness.append((EP[i].fitness[0],EP[i].fitness[1],EP[i].fitness[2], i))
import operator
fitness.sort(key = operator.itemgetter(0,1,2))
print("加急工件最大完成时间:",fitness[0][0])

best_idx = fitness[0][3]

print(f"基因：{EP[best_idx].CHS}")

print("产品完工耗时：",[EP[best_idx].JS.Jobs[i].endt
                 for i in range(len(EP[best_idx].JS.Jobs))])

# 绘制每代的最优目标以及自适应交叉变异图

# 创建一个新的图形
fig, ax1 = plt.subplots()

# 绘制第一个列表的折线图，使用左侧y轴
ax1.plot([ObjInfo[i][0] for i in range(len(ObjInfo))], '-', color='blue')
ax1.set_ylabel('最大完成时间')
ax1.set_xlabel('进化代数')
ax1.legend(['最大加工时间'],loc='center right',bbox_to_anchor=(1, 0.85))

# 创建第二个坐标轴
ax2 = ax1.twinx()
ax2.plot([ObjInfo[i][1] for i in range(len(ObjInfo))], '--', color='#00BFFF')
ax2.set_ylabel('机器空闲时间和', color='black')
ax2.legend(['机器停工时间和'],loc='upper right')

# 自动调整子图布局
plt.tight_layout()

# 保存图形
plt.savefig(f'q3(2)Obj-{fitness[0][0]}.png', dpi=300)

# 显示图形
plt.show()

# 设置工件和颜色对应关系
colorLst = ['#e6194B', '#3cb44b', '#ffe119', '#4363d8', 
            '#f58231', '#911eb4', '#42d4f4', '#f032e6', 
            '#bfef45', '#fabed4', '#469990', '#dcbeff', 
            '#9A6324', '#fffac8', '#800000', '#aaffc3', 
            '#808000', '#ffd8b1', '#000075', '#a9a9a9']

# 创建甘特图
fig, ax = plt.subplots(figsize=(10, 6))

# 循环遍历每个机器
for i, Machine in enumerate(EP[best_idx].JS.Machines):
    start_times = Machine.start
    end_times = Machine.end
    on = Machine._on
    
    # 循环遍历当前机器的工序信息
    for j, task_info in enumerate(on):
        workpiece = task_info[0]  # 工件编号
        operation = task_info[1]  # 工序编号

        # 获取工件对应的颜色
        color = colorLst[workpiece]

        # 绘制甘特图条形
        ax.broken_barh([(start_times[j], end_times[j]-start_times[j])], (i*10, 9), facecolors=color, edgecolor='white')
        
        # 计算亮度
        bg_color = mcolors.to_rgb(color)
        brightness = (0.299 * bg_color[0] + 0.587 * bg_color[1] + 0.114 * bg_color[2])

        # 根据亮度决定文本颜色
        if brightness < 0.5:
            text_color = 'white'
        else:
            text_color = 'black'

        # 添加工件序号标签
        if end_times[j]-start_times[j] > 4500:
            ax.text(start_times[j], i*10 + 5, str(workpiece + 1)+"-"+str(operation+1), ha='left', va='center', color=text_color, fontsize=14)
        # elif end_times[j]-start_times[j] > 2000:
        else:
            ax.text(start_times[j], i*10 + 5, str(workpiece + 1)+"-"+str(operation+1), ha='left', va='center', color=text_color, fontsize=9)

# 设置图形显示属性
# ax.set_ylim(0, len(EP[0].JS.Machines) * 10)
ax.set_xlabel('时间', fontsize=15)
ax.set_ylabel('机器', fontsize=15)
ax.set_yticks([i*10+5 for i in range(len(EP[0].JS.Machines))])
ax.set_yticklabels([chr(i) for i in range(ord('A'), ord('A') + m)], fontsize=18)  # 根据实际机器的名称进行调整
#ax.grid(True, axis='y', linestyle='--', linewidth=0.5)

# 隐藏顶部和右侧的边框
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
# 调整甘特图条形颜色的透明度
for collection in ax.collections:
    collection.set_alpha(1)

# 显示甘特图
plt.tight_layout()
plt.savefig(f'q3(2)甘特图-{fitness[0][0]}.png',dpi=400)

import pickle

# 保存EP、ObjInfo、pc和pm
data = {
    'EP': EP,
    'ObjInfo': ObjInfo,
    'pc': pc,
    'pm': pm
}

with open(f'q3(2)_best.pickle-{fitness[0][0]}', 'wb') as f:
    pickle.dump(data, f)


In [ ]:
while True:

    EP,ObjInfo,pc,pm=Algo.NSGA_main()

    ## 按照优先级排序
    fitness = []
    for i in range(len(EP)):
        fitness.append((EP[i].fitness[0],EP[i].fitness[1],EP[i].fitness[2], i))
    import operator
    fitness.sort(key = operator.itemgetter(0,1,2))
    print("加急工件最大完成时间:",fitness[0][0])

    best_idx = fitness[0][3]

    print(f"基因：{EP[best_idx].CHS}")

    print("产品完工耗时：",[EP[best_idx].JS.Jobs[i].endt
                    for i in range(len(EP[best_idx].JS.Jobs))])

    # 绘制每代的最优目标以及自适应交叉变异图

    # 创建一个新的图形
    fig, ax1 = plt.subplots()

    # 绘制第一个列表的折线图，使用左侧y轴
    ax1.plot([ObjInfo[i][0] for i in range(len(ObjInfo))], '-', color='blue')
    ax1.set_ylabel('最大完成时间')
    ax1.set_xlabel('进化代数')
    ax1.legend(['最大加工时间'],loc='center right',bbox_to_anchor=(1, 0.85))

    # 创建第二个坐标轴
    ax2 = ax1.twinx()
    ax2.plot([ObjInfo[i][1] for i in range(len(ObjInfo))], '--', color='#00BFFF')
    ax2.set_ylabel('机器空闲时间和', color='black')
    ax2.legend(['机器停工时间和'],loc='upper right')

    # 自动调整子图布局
    plt.tight_layout()

    # 保存图形
    plt.savefig(f'q3(2)Obj-{fitness[0][0]}.png', dpi=300)

    # 显示图形
    #plt.show()

    # 设置工件和颜色对应关系
    colorLst = ['#e6194B', '#3cb44b', '#ffe119', '#4363d8', 
                '#f58231', '#911eb4', '#42d4f4', '#f032e6', 
                '#bfef45', '#fabed4', '#469990', '#dcbeff', 
                '#9A6324', '#fffac8', '#800000', '#aaffc3', 
                '#808000', '#ffd8b1', '#000075', '#a9a9a9']

    # 创建甘特图
    fig, ax = plt.subplots(figsize=(10, 6))

    # 循环遍历每个机器
    for i, Machine in enumerate(EP[best_idx].JS.Machines):
        start_times = Machine.start
        end_times = Machine.end
        on = Machine._on
        
        # 循环遍历当前机器的工序信息
        for j, task_info in enumerate(on):
            workpiece = task_info[0]  # 工件编号
            operation = task_info[1]  # 工序编号

            # 获取工件对应的颜色
            color = colorLst[workpiece]

            # 绘制甘特图条形
            ax.broken_barh([(start_times[j], end_times[j]-start_times[j])], (i*10, 9), facecolors=color, edgecolor='white')
            
            # 计算亮度
            bg_color = mcolors.to_rgb(color)
            brightness = (0.299 * bg_color[0] + 0.587 * bg_color[1] + 0.114 * bg_color[2])

            # 根据亮度决定文本颜色
            if brightness < 0.5:
                text_color = 'white'
            else:
                text_color = 'black'

            # 添加工件序号标签
            if end_times[j]-start_times[j] > 4500:
                ax.text(start_times[j], i*10 + 5, str(workpiece + 1)+"-"+str(operation+1), ha='left', va='center', color=text_color, fontsize=14)
            # elif end_times[j]-start_times[j] > 2000:
            else:
                ax.text(start_times[j], i*10 + 5, str(workpiece + 1)+"-"+str(operation+1), ha='left', va='center', color=text_color, fontsize=9)

    # 设置图形显示属性
    # ax.set_ylim(0, len(EP[0].JS.Machines) * 10)
    ax.set_xlabel('时间', fontsize=15)
    ax.set_ylabel('机器', fontsize=15)
    ax.set_yticks([i*10+5 for i in range(len(EP[0].JS.Machines))])
    ax.set_yticklabels([chr(i) for i in range(ord('A'), ord('A') + m)], fontsize=18)  # 根据实际机器的名称进行调整
    #ax.grid(True, axis='y', linestyle='--', linewidth=0.5)

    # 隐藏顶部和右侧的边框
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    # 调整甘特图条形颜色的透明度
    for collection in ax.collections:
        collection.set_alpha(1)

    # 显示甘特图
    plt.tight_layout()
    plt.savefig(f'q3(2)甘特图-{fitness[0][0]}.png',dpi=400)

    import pickle

    # 保存EP、ObjInfo、pc和pm
    data = {
        'EP': EP,
        'ObjInfo': ObjInfo,
        'pc': pc,
        'pm': pm
    }

    with open(f'q3(2)_best.pickle-{fitness[0][0]}', 'wb') as f:
        pickle.dump(data, f)
